# Create reusable programmatic units that can be composed into dashboards and used in notebooks.

In [1]:
    import traitlets, pandas, toolz.curried as toolz, ipywidgets, json, typing, pathlib, IPython, lsst.daf.persistence, lsst.qa.explorer.functors, datashader, matplotlib, operator, param, holoviews.operation.datashader, bokeh

Subselect some parameters.  We can pick and choose which we prefer.

In [2]:
    metric: ['base_Footprint_nPix', 'Gaussian-PSF_magDiff_mmag', 'CircAper12pix-PSF_magDiff_mmag', 'Kron-PSF_magDiff_mmag', 'CModel-PSF_magDiff_mmag', 'traceSdss_pixel', 'traceSdss_fwhm_pixel', 'psfTraceSdssDiff_percent', 'e1ResidsSdss_milli', 'e2ResidsSdss_milli', 'deconvMoments', 'compareUnforced_Gaussian_magDiff_mmag', 'compareUnforced_CircAper12pix_magDiff_mmag', 'compareUnforced_Kron_magDiff_mmag', 'compareUnforced_CModel_magDiff_mmag'] 
    metric: __annotations__['metric'][:10]
    tract: [9615, 9813]
    tract = __annotations__['tract'][0]
    flag: ['calib_psf_used', 'calib_psf_candidate', 'calib_photometry_reserved', 'merge_measurement_i2', 'merge_measurement_i', 'merge_measurement_r2', 'merge_measurement_r', 'merge_measurement_z', 'merge_measurement_y', 'merge_measurement_g', 'merge_measurement_N921', 'merge_measurement_N816', 'merge_measurement_N1010', 'merge_measurement_N387', 'merge_measurement_N515', 'qaBad_flag']
    flag: __annotations__['flag'][:10]
    filter: ['HSC-G', 'HSC-I', 'HSC-R', 'HSC-Z', 'HSC-Y'] 
    filter = __annotations__['filter'][1]
    table: ['analysisCoaddTable_forced']
    table = __annotations__['table'][0]
    target: ['/project/tmorton/tickets/DM-20015/RC2_w18'] 
    target = __annotations__['target'][0]
    composite: typing.Dict[str, lsst.qa.explorer.functors.Functor] = {'label': lsst.qa.explorer.functors.StarGalaxyLabeller(), 'ra': lsst.qa.explorer.functors.RAColumn(), 'dec': lsst.qa.explorer.functors.DecColumn()}

Connect to parquest to access our data.

In [10]:
    conn = lsst.daf.persistence.Butler(target)
    match = conn.get('visitMatchTable', tract=tract, filter=filter).toDataFrame()['matchId']

In [11]:
    __name__ == '__main__' and display(match.T.iloc[:10, :10], metadata={'match': json.loads(match.head(0).to_json(orient='table'))})

id,42287217204264961,42287217204264962,42287217204264963,42287217204264964,42287217204264965,42287217204264966,42287217204264967,42287217204264968,42287217204264969,42287217204264970
1280,1.099649e+15,1.099649e+15,1.099649e+15,1.099649e+15,1.099649e+15,1.099649e+15,1.099649e+15,1.099649e+15,1.099649e+15,1.099649e+15
1282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1290,1.108162e+15,1.108162e+15,1.108162e+15,1.108162e+15,1.108162e+15,1.108162e+15,1.108162e+15,1.108162e+15,NaN,1.108166e+15
24522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1302,1.118620e+15,1.118620e+15,1.118620e+15,1.118620e+15,1.118620e+15,1.118620e+15,1.118620e+15,1.118620e+15,1.118620e+15,1.118620e+15
24536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
    parquet = conn.get(table, tract=tract, filter=filter)
    raw = pandas.concat([
        parquet.toDataFrame(columns=__annotations__['metric']+__annotations__['flag']), 
        lsst.qa.explorer.functors.CompositeFunctor(composite)(parquet)
    ], axis=1)
    raw[__annotations__['flag']] = raw[__annotations__['flag']].astype(int)
    raw = raw.set_index(list(composite.keys())+__annotations__['flag'], append=True)
    if __name__ == '__main__':
        display(raw.T, metadata={'raw': json.loads(raw.head(0).to_json(orient='table'))})

id,42287217204264961,42287217204264962,42287217204264963,42287217204264964,42287217204264965,42287217204264966,42287217204264967,42287217204264968,42287217204264969,42287217204264970,...,42288351075661679,42288351075661680,42288351075661681,42288351075661682,42288351075661683,42288351075661684,42288351075661685,42288351075661686,42288351075661687,42288351075661688
label,null,null,null,null,null,null,null,null,null,null,...,null,null,null,null,star,null,galaxy,star,star,galaxy
ra,217.132928,217.125231,217.113521,217.112401,217.101904,217.098266,217.095513,217.092807,217.078018,217.060337,...,215.500755,215.501304,215.647176,215.646990,215.549320,215.549725,215.530716,215.531105,215.530274,215.531493
dec,-0.095521,-0.095848,-0.095755,-0.095662,-0.095849,-0.095849,-0.095756,-0.095429,-0.095849,-0.095663,...,1.419002,1.418999,1.422165,1.422025,1.439279,1.439212,1.440442,1.440464,1.440535,1.440431
calib_psf_used,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
calib_psf_candidate,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
calib_photometry_reserved,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
merge_measurement_i2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
merge_measurement_i,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
merge_measurement_r2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
merge_measurement_r,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
    visits = pandas.concat({
        object: conn.get('analysisVisitTable', visit=object, tract=tract, filter=filter).toDataFrame(columns=__annotations__['metric']) 
        for object in match.loc[raw.index.get_level_values('id')].pipe(
            lambda df: df[df.any(axis=1)]
        ).columns})
    if __name__ == '__main__':
        display(visits.T, metadata={'visits': json.loads(visits.head(0).to_json(orient='table'))})


1258                     \
id                             1080845699907585  1080845699907586    
base_Footprint_nPix                   315.000000         41.000000   
Gaussian-PSF_magDiff_mmag            -206.819345       -685.231110   
CircAper12pix-PSF_magDiff_mmag               NaN               NaN   
Kron-PSF_magDiff_mmag                        NaN               NaN   
traceSdss_pixel                         1.659893         77.390163   
traceSdss_fwhm_pixel                    3.908750        182.239906   
psfTraceSdssDiff_percent               19.751169        193.076311   
e1ResidsSdss_milli                    -52.130269        950.151015   
e2ResidsSdss_milli                    -15.569402       -253.137706   

                                                                    \
id                             1080845699907587  1080845699907588    
base_Footprint_nPix                    36.000000         93.000000   
Gaussian-PSF_magDiff_mmag             378.604393         44.862122   
CircAper12pix-PSF_magDiff_mmag               NaN               NaN   
Kron-PSF_magDiff_mmag                        NaN               NaN   
traceSdss_pixel                        83.287455          2.453892   
traceSdss_fwhm_pixel                  196.126968          5.778475   
psfTraceSdssDiff_percent              193.558127         56.907778   
e1ResidsSdss_milli                    959.161386       -146.677318   
e2ResidsSdss_milli                   -268.903002       -460.268647   

                                                                    \
id                             1080845699907589  1080845699907590    
base_Footprint_nPix                    63.000000       1122.000000   
Gaussian-PSF_magDiff_mmag            -890.872261       -689.780782   
CircAper12pix-PSF_magDiff_mmag               NaN               NaN   
Kron-PSF_magDiff_mmag                        NaN               NaN   
traceSdss_pixel                         4.032697          2.459004   
traceSdss_fwhm_pixel                    9.496276          5.790512   
psfTraceSdssDiff_percent               98.732585         57.067333   
e1ResidsSdss_milli                    185.578986       -321.955980   
e2ResidsSdss_milli                   -204.354448        -62.038125   

                                                                    \
id                             1080845699907591  1080845699907592    
base_Footprint_nPix                   444.000000        104.000000   
Gaussian-PSF_magDiff_mmag                    NaN       -255.505501   
CircAper12pix-PSF_magDiff_mmag       -471.464349               NaN   
Kron-PSF_magDiff_mmag                -452.947753       -186.300884   
traceSdss_pixel                         4.633819          1.712854   
traceSdss_fwhm_pixel                   10.911809          4.033463   
psfTraceSdssDiff_percent              108.838608         22.386908   
e1ResidsSdss_milli                   -332.083058         84.293010   
e2ResidsSdss_milli                    730.512345       -294.766688   

                                                                    ...  \
id                             1080845699907593  1080845699907594   ...   
base_Footprint_nPix                   210.000000        683.000000  ...   
Gaussian-PSF_magDiff_mmag            -545.072272       -635.345602  ...   
CircAper12pix-PSF_magDiff_mmag       -942.877067       -847.396385  ...   
Kron-PSF_magDiff_mmag                -713.095223       -736.442595  ...   
traceSdss_pixel                         2.252122          2.265475  ...   
traceSdss_fwhm_pixel                    5.303342          5.334785  ...   
psfTraceSdssDiff_percent               48.888725         49.578694  ...   
e1ResidsSdss_milli                   -359.456520        -26.668015  ...   
e2ResidsSdss_milli                      6.937365       -184.519734  ...   

                                           24538                    \
id                             21078423883482875 21078423883482

In [21]:
    if __name__ == '__main__':
        raw.to_hdf(F'{tract}.h5', 'raw')
        psf.to_hdf(F'{tract}.h5', 'psf')
        psf.to_hdf(F'{tract}.h5', 'visits')

In [22]:
    !ls -l 9615.h5

-rw-r--r-- 1 tonyfast tonyfast 1036054208 Jul 10 16:35 9615.h5


In [23]:
    __name__ == '__main__' and pandas.read_hdf(F'{tract}.h5', 'raw').sample(10).T

id,42287513557010114,42287358938192704,42287659585925913,42288351075654279,42287363233158595,42287367528150485,42287230089195285,42288200751785253,42287358938218080,42288063312835299
label,galaxy,galaxy,star,galaxy,galaxy,galaxy,galaxy,galaxy,star,galaxy
ra,216.647859,216.892225,216.482544,215.568330,216.789325,216.841665,217.103674,215.703444,216.925263,215.868731
dec,0.861463,0.189903,1.264594,1.443509,0.355372,0.486182,0.474859,0.981931,0.142538,1.025651
calib_psf_used,0,0,0,0,0,0,0,0,1,0
calib_psf_candidate,0,0,0,0,0,0,0,0,1,0
calib_photometry_reserved,0,0,0,0,0,0,0,0,0,0
merge_measurement_i2,0,0,0,0,0,0,0,0,0,0
merge_measurement_i,1,1,1,1,1,1,1,1,1,1
merge_measurement_r2,0,0,0,0,0,0,0,0,0,0
merge_measurement_r,0,0,0,0,0,0,0,0,0,0


In [24]:
    __name__ == '__main__' and pandas.read_hdf(F'{tract}.h5', 'psf').sample(10).T

id,42287242974098574,42287513557043330,42288067607803875,42287238679102205,42288196456814650,42288316715927154,42287217204276793,42287938758792389,42288050427928924,42287629521126572
label,galaxy,galaxy,star,galaxy,galaxy,galaxy,null,galaxy,galaxy,galaxy
ra,216.948689,216.675930,215.996474,216.957518,215.669505,215.606128,216.998317,216.087004,215.865496,216.547084
dec,1.051646,0.938595,1.173769,0.845802,0.742351,0.011791,0.094782,1.400491,0.409378,-0.077695
calib_psf_used,0,0,0,0,0,0,0,0,0,0
calib_psf_candidate,0,0,0,0,0,0,0,0,0,0
calib_photometry_reserved,0,0,0,0,0,0,0,0,0,0
merge_measurement_i2,0,0,0,0,0,0,0,0,0,0
merge_measurement_i,1,0,0,0,1,1,1,1,1,1
merge_measurement_r2,0,0,0,0,0,0,0,0,0,0
merge_measurement_r,0,1,0,1,0,0,0,0,0,0


In [25]:
    __name__ == '__main__' and pandas.read_hdf(F'{tract}.h5', 'visits').sample(10).T

id,42288351075638531,42288321010872077,42287526441935722,42287230089169448,42288316715922759,42287788434951287,42288071902762732,42287225794207708,42287251564007799,42288325305832830
label,star,galaxy,galaxy,galaxy,galaxy,galaxy,galaxy,galaxy,galaxy,star
ra,215.586810,215.610814,216.619574,217.052988,215.534940,216.260917,215.903931,217.133524,217.026084,215.613197
dec,1.560154,0.268240,1.409794,0.501354,-0.045898,0.974324,1.331019,0.401895,1.480659,0.356542
calib_psf_used,0,0,0,0,0,0,0,0,0,0
calib_psf_candidate,0,0,0,0,0,0,0,0,0,0
calib_photometry_reserved,0,0,0,0,0,0,0,0,0,0
merge_measurement_i2,0,0,0,0,0,0,0,0,0,0
merge_measurement_i,0,1,1,1,1,1,1,1,1,1
merge_measurement_r2,0,0,0,0,0,0,0,0,0,0
merge_measurement_r,0,0,0,0,0,0,0,0,0,0


In [26]:
    if __name__ == '__main__':
        !zip tracts.zip 9615.h5

updating: 9615.h5 (deflated 49%)


In [25]:
    __name__ == '__main__' and \
    !ls -l tracts.zip

-rw-r--r-- 1 tonyfast tonyfast 325043036 Jul 10 16:23 tracts.zip
